In [25]:
import torch
from transformers import *
import pandas as pd
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
batch = str(0)
sent = str(0)
batch_0_x_sent_0 = torch.load('tensor/batch_{}_x_sent_{}.pt'.format(batch, sent)).squeeze(0)
batch_0_hess_0 = torch.load('tensor/batch_{}_hess_{}.pt'.format(batch, sent)).squeeze(0)
# print(batch_0_x_sent_0)

tensor([  101,  2048,  2308,  2024, 23581,  2096,  3173,  2000,  2175, 14555,
         1012,   102,  1996,  5208,  2024, 17662,  9119,  2096,  3173,  2000,
         2175, 14555,  2044,  2074,  5983,  6265,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
       device='cuda:0')


In [45]:
ori_sentence = tokenizer.convert_ids_to_tokens(batch_0_x_sent_0)
# print(tokenizer.convert_ids_to_tokens(batch_0_x_sent_0))
temp = dict([i, ori_sentence[i]] for i in range(len(ori_sentence)))

In [7]:
print(torch.abs(batch_0_hess_0))
abs_hess = torch.abs(batch_0_hess_0)
# no_diag_hess = abs_hess - abs_hess.max() * torch.eye(abs_hess.shape[0], device='cuda:0')

tensor([[0.0642, 0.0008, 0.0380,  ..., 0.0000, 0.0000, 0.0000],
        [0.0008, 0.0507, 0.0255,  ..., 0.0000, 0.0000, 0.0000],
        [0.0380, 0.0255, 0.0331,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')


In [11]:
# abs_hess.split(128)
# conv = torch.nn.Conv2d(1, 1, (128, 128), padding=0, stride=128).to('cuda:0')(abs_hess.unsqueeze(0).unsqueeze(0))
avg_hess = torch.nn.AvgPool2d(128, stride=128).to('cuda:0')(abs_hess.unsqueeze(0))[0] - torch.eye(40, device='cuda:0') * 10
max_hess = torch.nn.MaxPool2d(128, stride=128).to('cuda:0')(abs_hess.unsqueeze(0))[0] - torch.eye(40, device='cuda:0') * 10

In [59]:
values, indices = max_hess.max(1)
pairs = [[i, int(indices[i]), float(values[i])] for i in range(indices.shape[0]) if float(values[i]) != 0]

In [66]:
df = pd.DataFrame(pairs)
df.columns = ['word', 'most relavent', 'score']
df = df.replace(temp)
df.sort_values('score', ascending=False, inplace=True)
df.head(10)

,word,most relavent,score
27,[SEP],[SEP],0.561189
11,[SEP],[SEP],0.561189
0,[CLS],[SEP],0.497816
21,packages,packages,0.199171
9,packages,packages,0.199171
2,women,[SEP],0.170174
13,sisters,[SEP],0.159542
10,.,[SEP],0.124365
4,embracing,[SEP],0.124075
26,.,[SEP],0.115160


In [18]:
values, indices = no_diag_hess.max(1)

In [13]:
# torch.topk(no_diag_hess, 5)

torch.return_types.topk(
values=tensor([[0.1602, 0.1550, 0.1335, 0.1164, 0.1150],
        [0.1623, 0.0946, 0.0881, 0.0861, 0.0788],
        [0.4964, 0.4913, 0.4907, 0.4043, 0.3721],
        ...,
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], device='cuda:0'),
indices=tensor([[3459, 3535, 3580, 3529, 1411],
        [   3,  112,    5, 3535,  118],
        [   3, 3459, 3535, 1411, 3529],
        ...,
        [   1,    0,    2,    4,    3],
        [   1,    0,    2,    4,    3],
        [   1,    0,    2,    4,    3]], device='cuda:0'))